In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

data = xr.open_dataset("../1_data_processing/1_1_Forcing/all_stations_cmems.zarr", engine="zarr")
data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
data = data.sel(time=slice(time_start, time_end))
_ = data.load()

cafe_npp = xr.open_dataset("../1_data_processing/1_1_Forcing/all_stations_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time", how="all")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
_ = cafe_npp.load()

In [3]:
data

<xarray.Dataset> Size: 13MB
Dimensions:              (time: 8767, depth: 3, latitude: 3, longitude: 3)
Coordinates:
  * depth                (depth) float32 12B 1.0 2.0 3.0
  * latitude             (latitude) float64 24B 22.75 31.6 50.0
  * longitude            (longitude) float64 24B -158.0 -145.0 -64.2
  * time                 (time) datetime64[ns] 70kB 1998-01-01 ... 2022-01-01
Data variables: (12/13)
    T                    (time, depth, latitude, longitude) float64 2MB 23.54...
    U                    (time, depth, latitude, longitude) float64 2MB 0.015...
    V                    (time, depth, latitude, longitude) float64 2MB -0.07...
    mnkc_epi             (time, latitude, longitude) float64 631kB 0.2627 ......
    mnkc_hmlmeso         (time, latitude, longitude) float64 631kB 0.7987 ......
    mnkc_lmeso           (time, latitude, longitude) float64 631kB 2.157 ... nan
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float64 631kB 0.4632 ......
    mnkc_umeso           (time, latitude, longitude) float64 631kB 0.4843 ......
    npp                  (time, latitude, longitude) float64 631kB 316.0 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 2MB 117.3...
    zeu                  (time, latitude, longitude) float64 631kB 78.17 ... nan
    zooc                 (time, latitude, longitude) float64 631kB 0.2832 ......
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2022-09-28
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_202211
    subset:date:       2024-06-12T03:45:47.386Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Global ocean low and mid trophic levels biomass conten...

In [4]:
cafe_npp

<xarray.Dataset> Size: 701kB
Dimensions:    (time: 8767, latitude: 3, longitude: 3)
Coordinates:
  * latitude   (latitude) float64 24B 22.75 31.6 50.0
  * longitude  (longitude) float64 24B -158.0 -150.0 -64.2
  * time       (time) datetime64[ns] 70kB 1998-01-01 1998-01-02 ... 2022-01-01
Data variables:
    CAFE       (time, latitude, longitude) float64 631kB 306.4 nan ... nan nan
Attributes:
    Conventions:  CF-1.8
    History:      2023-01-02 08:55:09.514113 Python
    Keywords:     Primary Production
    PI Contact:   tryankeogh@csir.co.za
    PI Name:      Thomas Ryan-Keogh

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [5]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../1_data_processing/1_1_Forcing/Bats_obs.zarr", engine="zarr")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)

obs_hot = xr.open_dataset("../1_data_processing/1_1_Forcing/Hot_obs.zarr", engine="zarr")
obs_hot = update_layer(obs_hot, epipelagic_size)
obs_hot = Observation(obs_hot)

obs_papa = xr.open_dataset("../1_data_processing/1_1_Forcing/Papa_obs.zarr", engine="zarr")
obs_papa = update_layer(obs_papa, epipelagic_size)
obs_papa = Observation(obs_papa)

observations = [obs_bats, obs_hot, obs_papa]

Create structure for SeapoPym simulation.


In [6]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=data["T"], resolution=1 / 12, timestep=1),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12, timestep=1),
)

2024-10-16 21:48:18,157 :: Seapodym ::  WARNING ::
|	CAFE unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



Setup the cost function.


In [7]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        tr_max=Parameter("D1N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        tr_max=Parameter("D2N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [8]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [9]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    ETA=1,
    INDPB=0.1,
    CXPB=0.5,
    MUTPB=0.2,
    NGEN=10,
    POP_SIZE=100,
    cost_function_weight=(-(1 / 3), -(1 / 3), -(1 / 3)),
    hall_of_fame_size=300,
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [10]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [11]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

And watch the magic on the Dask dashboard :


In [12]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51557,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:51570,Total threads: 2
Dashboard: http://127.0.0.1:51572/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:51560,


And execute the process.


In [13]:
viewer = genetic_algo.optimize()

2024-10-16 21:48:20,059 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-10-16 21:48:20,062 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-10-16 21:48:20,063 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-10-16 21:48:20,065 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-10-16 21:48:20,065 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-10-16 21:48:20,066 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-10-16 21:48:20,067 :: Seapodym ::  DEBUG ::
|	Direct computation for global_mask_from_nan.

2024-10-16 21:48:20,071 :: Seapodym ::  DEBUG ::
|	Direct computation for mask_by_fgroup.

2024-10-16 21:48:20,075 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-10-16 21:48:20,078 :: Seapodym ::  DEBUG ::
|	Direct computation for _wrapper_mesh_day_lengths.

2024-10-16 21:48:20,084 :: Seapodym ::  DEBU

In [14]:
viewer.logbook

,gen,nevals,avg,std,min,max,nvalide,ninvalide
0,0,100,0.000018,0.000031,1.332802e-07,0.000323,192,108
1,1,68,0.000016,0.000028,1.252992e-07,0.000315,276,24
2,2,59,0.000014,0.000019,1.252992e-07,0.000042,285,15
3,3,71,0.000014,0.000019,1.252992e-07,0.000042,285,15
4,4,62,0.000014,0.000019,1.252992e-07,0.000042,294,6
5,5,82,0.000014,0.000019,1.252992e-07,0.000042,297,3
6,6,63,0.000015,0.000019,1.212027e-07,0.000050,300,0
7,7,64,0.000014,0.000019,1.212027e-07,0.000042,285,15
8,8,55,0.000014,0.000019,1.212027e-07,0.000042,288,12
9,9,64,0.000014,0.000019,1.208507e-07,0.000042,288,12


Finaly here is the result :


In [15]:
viewer.box_plot(4)

In [16]:
viewer.parallel_coordinates()